In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure , output_notebook, show
%matplotlib inline
output_notebook()

Loading BokehJS ...

In [2]:
mush = pd.read_csv("mushrooms.csv")

In [3]:
mush.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [4]:
mush.head()
for i in mush.columns:
    print("{} has ---{}--- unique values which are :{}".format(i , mush[i].nunique(),mush[i].unique()))


class has ---2--- unique values which are :['p' 'e']
cap-shape has ---6--- unique values which are :['x' 'b' 's' 'f' 'k' 'c']
cap-surface has ---4--- unique values which are :['s' 'y' 'f' 'g']
cap-color has ---10--- unique values which are :['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
bruises has ---2--- unique values which are :['t' 'f']
odor has ---9--- unique values which are :['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
gill-attachment has ---2--- unique values which are :['f' 'a']
gill-spacing has ---2--- unique values which are :['c' 'w']
gill-size has ---2--- unique values which are :['n' 'b']
gill-color has ---12--- unique values which are :['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
stalk-shape has ---2--- unique values which are :['e' 't']
stalk-root has ---5--- unique values which are :['e' 'c' 'b' 'r' '?']
stalk-surface-above-ring has ---4--- unique values which are :['s' 'f' 'k' 'y']
stalk-surface-below-ring has ---4--- unique values which are :['s' 'f' 'y' 'k']
stalk-color-ab

In [5]:
from pivottablejs import pivot_ui
pivot_ui(mush)

In [6]:
mush.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [7]:
X = pd.get_dummies(mush.iloc[:,1:],drop_first=True)
y = pd.get_dummies(mush.iloc[: , 0],drop_first=True)

In [8]:
X = X.iloc[:,:].values

In [9]:
y = y.iloc[:,0].values

In [10]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train,y_test = train_test_split(X, y , test_size = 0.15 , random_state = 42)

In [11]:
assert(X.shape[1]==X_train.shape[1]==X_test.shape[1])

## XGBoost Algorithm

In [12]:
from xgboost import XGBClassifier
classifier = XGBClassifier(n_estimators = 100 , max_depth= 2)
classifier.fit(X_train , y_train)

C:\Users\Mostafa\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [13]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test , y_pred)
score

0.99917965545529119

In [14]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier ,X = X , y = y , scoring = "accuracy", cv=10)
accuracies.mean()

0.96136244716331265

In [15]:
print("XGB classified the Mushroom problem with kfold mean of {}+-{}".format(accuracies.mean(),accuracies.std()))

XGB classified the Mushroom problem with kfold mean of 0.9613624471633127+-0.09354660680536528


### XGB classified the Mushroom problem with kfold mean of 0.9613624471633127 +-  0.09354660680536528

### Stacked Classifier

In [46]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

c1 = KNeighborsClassifier(n_neighbors=1)
c2 = RandomForestClassifier(random_state=1)
c3 = GaussianNB()
c4 = SVC()
lr = LogisticRegression()


for c , label in zip([c1,c2,c3,c4,lr],["KNN","RandomForest","Naive Bayes","Support Vector Machine","Linear Reg"]):
    scores = cross_val_score(c, X, y , cv = 10, scoring = "accuracy")
    print("accuracy {} +/- {} [{}]".format(scores.mean() , scores.std(), label))

accuracy 0.9691266912669126 +/- 0.09261992619926201 [KNN]
accuracy 0.9685116851168513 +/- 0.09446494464944649 [RandomForest]
accuracy 0.8882360571068249 +/- 0.08177592542721115 [Naive Bayes]
accuracy 0.9368360625653113 +/- 0.11254858565593036 [Support Vector Machine]
accuracy 0.9542098725549524 +/- 0.0984655929427475 [Linear Reg]


### Genetic Algorithm

In [28]:
from tpot import TPOTClassifier
tpot_classifier = TPOTClassifier(generations=5,population_size=60,verbosity=2,max_time_mins=2)
tpot_classifier.fit(X,y)

TPOT closed prematurely. Will use the current best pipeline.



Best pipeline: RandomForestClassifier(KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=8, KNeighborsClassifier__p=1, KNeighborsClassifier__weights=uniform), RandomForestClassifier__bootstrap=True, RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_features=0.15, RandomForestClassifier__min_samples_leaf=20, RandomForestClassifier__min_samples_split=19, RandomForestClassifier__n_estimators=DEFAULT)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        generations=1000000, max_eval_time_mins=5, max_time_mins=2,
        mutation_rate=0.9, n_jobs=1, offspring_size=60, population_size=60,
        random_state=None, scoring=None, subsample=1.0, verbosity=2,
        warm_start=False)

In [29]:
tpot_classifier.score(X_test , y_test)

1.0

In [30]:
tpot_classifier.export("Mush_t.py")

In [37]:
# %load Mush_t.py
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

# # NOTE: Make sure that the class is labeled 'class' in the data file
# tpot_data = np.recfromcsv('PATH/TO/DATA/FILE', delimiter='COLUMN_SEPARATOR', dtype=np.float64)
# features = np.delete(tpot_data.view(np.float64).reshape(tpot_data.size, -1), tpot_data.dtype.names.index('class'), axis=1)
training_features, testing_features, training_target, testing_target = \
    train_test_split(X,y, random_state=42, test_size = 0.15)

exported_pipeline = make_pipeline(
    StackingEstimator(estimator=KNeighborsClassifier(n_neighbors=8, p=1, weights="uniform")),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.15000000000000002, min_samples_leaf=20, min_samples_split=19)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


### We can see the perfect score of 100 % by comparing the Y test with the predicted results

In [38]:
accuracy_score(results, y_test)

1.0

### and a perfect score of 10 fold cross validation

In [40]:
scores =cross_val_score(exported_pipeline,X_train , y_train , cv = 10 , scoring = "accuracy")
print(scores.mean() , scores.std())

1.0 0.0
